In [7]:
import pymongo
from pymongo import MongoClient
import json
#from pprint import pprint
from pathlib import Path

#Création d'une connexion au serveur mongoDB.
def get_client():

    try:
        client = MongoClient(host = "mongodb://localhost:27017/")
        #client.server_info()
    except:
        print ("connection error")

    return client

In [13]:
################################ insertOne() avec MongoDB & Python (pymongo) ################################


#Q : Créer une première DB "metflix" qui contient une collection "movies" dans laquelle vous inserez les documents suivants : 
# Comparer le comportement du shell Mongo et Python

def Q1():
    #Connexion à mongo...
    my_client = get_client()
    
    #Suppression (si elle existe) puis création de la base de donnée.
    if "metflix" in my_client.list_database_names():
        print("The database exists.")
        my_client.drop_database('metflix')
        
        #Création de la base de donnée.
        my_db = my_client["metflix"]
    
        #Création des collections et insertions des données.
        if "movies" or "users" not in my_db.list_collection_names():
            print("Collections don't exist, database has been dropped...")
            new_col1 = my_db["movies"]
            data_list = [
                { "title": "Rocky", "year": 1976},
                { "title": "Rocky", "year": 1976,  "_id" : "tt0075148"},
                { "firstname": "Charlon", "lastname": "Owain"}
            ]
            new_col1.insert_one(data_list[0])
            new_col1.insert_one(data_list[1])

            new_col2 = my_db["users"]
            new_col2.insert_one(data_list[2])
            print("Collections created with success.")
    
Q1()

The database exists.
Collections don't exist, database has been droped...


In [19]:
################################ insertMany() ################################

# Q : Supp la dernière collection, la recréer en y insérant les documents suivants :
#   title = Rocky, year = 1976, _id = tt0075148
#   title = Jaws, year = 1975, imdb = tt0073195
#   title = Mad Max 2 : The Road Warrior, year = 1981, imdb = tt0082694
#   title = Raiders of the Lost Ark, year = 1981, imdb = tt0082971

def Q2():
    #Connexion à mongo...
    my_client = get_client()
    
    #Si la base de données existe...
    if "metflix" in my_client.list_database_names():
        print("The database exists.")
        
        #Connexion à la base de donnée.
        my_db = my_client["metflix"]
        
        #Suppression de la dernière collection si elle existe.
        if "movies" and "users" in my_db.list_collection_names():
            movies_col = my_db["movies"]
            movies_col.drop()
        #Création de la nouvelle collection et insertion des données via insertMany()
        new_movies_col = my_db["movies"]
        
        data_list = [
                { "title": "Rocky", "year": 1976, "_id" : "tt0075148"},
                { "title": "Jaws", "year": 1975,  "imdb" : "tt0073195"},
                { "title": "Mad Max 2 : The Road Warrior", "year": 1981,  "imdb" : "tt0082694"},
                { "title": "Raiders of the Lost Ark", "year": 1981,  "imdb" : "tt0082971"}
        ]
        
        new_movies_col.insert_many(data_list)
        print("Collection created with success.")
    else:
        print("The database doesn't exist...You should create it by calling previous methods")

Q2()    

The database exists.
Collection created with success.


In [11]:
################################ find() avec des paramètres ################################

# Q : Afficher ts les documents de la collection movies, le tout 1er document, 
# le(s) document(s) où on trouve les films produits en 1981, le document où on trouve le film qui porte le titre "Jaws".

def Q3():
    #Connexion à mongo...
    my_client = get_client()
    
    #Si la base de données existe...
    if "metflix" in my_client.list_database_names():
        print("The database exists.")
        
        #Connexion à la base de donnée et à la collection movies.
        my_db = my_client["metflix"]
        movies_col = my_db["movies"]
        
        #Récuperation et affichage de tous les documents de la collection movies.
        all_movies = [movie for movie in movies_col.find()]
        print( 'All movies:\n{}'.format(all_movies) )
        
        #Récuperation et affichage du tout 1er document.
        first_movie = movies_col.find_one()
        print('First movie:\n{}'.format(first_movie))
        
        #Récuperation et affichage du (ou des) document(s) où se trouve les films produits en 1981.
        year_movies = [movie for movie in movies_col.find({"year": 1981})]
        print('1981 movie(s):\n{}'.format(year_movies))
        
        #Récuperation et affichage du document où se trouve le dont le titre est "Jaws".
        title_movie = [movie for movie in movies_col.find({"title": "Jaws"})]
        print('Movie named Jaws:\n{}'.format(title_movie))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")

Q3()

The database exists.
All movies:
[{'_id': 'tt0075148', 'title': 'Rocky', 'year': 1976}, {'_id': ObjectId('60031fc78beb27dff80c4137'), 'title': 'Jaws', 'year': 1975, 'imdb': 'tt0073195'}, {'_id': ObjectId('60031fc78beb27dff80c4138'), 'title': 'Mad Max 2 : The Road Warrior', 'year': 1981, 'imdb': 'tt0082694'}, {'_id': ObjectId('60031fc78beb27dff80c4139'), 'title': 'Raiders of the Lost Ark', 'year': 1981, 'imdb': 'tt0082971'}]
"First movie:\n{'_id': 'tt0075148', 'title': 'Rocky', 'year': 1976}"
"1981 movie(s):\n[{'_id': ObjectId('60031fc78beb27dff80c4138'), 'title': 'Mad Max 2 : The Road Warrior', 'year': 1981, 'imdb': 'tt0082694'}, {'_id': ObjectId('60031fc78beb27dff80c4139'), 'title': 'Raiders of the Lost Ark', 'year': 1981, 'imdb': 'tt0082971'}]"
"Movie named Jaws:\n[{'_id': ObjectId('60031fc78beb27dff80c4137'), 'title': 'Jaws', 'year': 1975, 'imdb': 'tt0073195'}]"


In [6]:
################################ Import de fichiers .json ################################

# Q : Créer une DB movies_artists avec deux collections : une pour les films et l’autre pour les artistes 
# (voir les fichiers artists.json et movies.json associés).
# Rq: Préciser, lors de l'injection des fichiers json, dans la commande "mongoimport" la possibilité de supprimer la collection 
# si elle existe (pour qu'elle soit recréer de nouveau et y injecter de la data fraîche).
# Indice :
#   use movies_artists  // La création de la BD est implicite
#   // La création des deux collections est implicites
#   mongoimport -d nom_db -c nom_collection --drop --file "C:/.../movies.json" --jsonArray
#   // Vérification
#   db.artists.find().pretty()
#   db.movies.find().pretty()

def Q4():
    #Connexion à mongo...
    my_client = get_client()
    
    #Suppression (si elle existe) puis création de la base de donnée, cela supprime aussi les collections à chaque fois...
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_client.drop_database('movies_artists')
        
    #Création de la base de donnée.
    my_db = my_client["movies_artists"]
        
    #Création des collections nécéssaires.
    new_col1 = my_db["artists"]
    new_col2 = my_db["movies"]
        
    #Insertion des données... Changer le chemin absolu si nécéssaire...
    data_artists_path = r"C:\Users\owcha\Desktop\NoSQL\ressources\json\artists.json" #raw string treats \ as a literal character.
    data_movies_path = r"C:\Users\owcha\Desktop\NoSQL\ressources\json\movies.json"
    try :
        with open(data_artists_path) as json_data:
            artists_data = json.load(json_data)

        with open(data_movies_path) as json_data:
            movies_data = json.load(json_data)
    except :
        print("Something went wrong, check your paths")
            
    new_col1.insert_many(artists_data)
    new_col2.insert_many(movies_data)
        
    print("Datas from json document well injected...")

Q4()

The database exists.
Datas from json document well injected...


In [7]:
# Q : Augmenter la collection movies et artists avec les fichiers movies_suite.json et artists_suite.json 
# après les avoir transformer en csv (la conversion du json vers csv est obligatoire).
# Indice :
# - à l'aide de mongoimport
# mongoimport -d movies_artists -c artists --type csv --headerline --file "C:/.../artists_suite.csv"
# mongoimport -d movies_artists -c collection movies --mode insert --file "C:/.../movies_suite.json" --jsonArray
# - Autres méthodes sont possibles : ...

def Q5():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        
        #Completion des collections si elles existent...
        if "artists" and "movies" in my_db.list_collection_names():
            #Insertion des données... Changer le chemin absolu si nécéssaire...
            data_artists_path2 = r"C:\Users\owcha\Desktop\NoSQL\ressources\json\artists_suite.json"
            data_movies_path2 = r"C:\Users\owcha\Desktop\NoSQL\ressources\json\movies_suite.json"
            try :
                with open(data_artists_path2) as json_data:
                    artists_data2 = json.load(json_data)

                with open(data_movies_path2) as json_data:
                    movies_data2 = json.load(json_data)
            except :
                print("Something went wrong, check your paths")
            
            my_db["artists"].insert_many(artists_data2)
            my_db["movies"].insert_many(movies_data2)
        
            print("Datas from json document well completed...")
        else:
            print("Collections don't exist...You should create it by calling previous methods")
    
    else:
        print("The database doesn't exist...You should create it by calling previous methods")

Q5()

# Q : Approfondissement
# Q : Qu'est-ce q'un upsert et cmnt faire upsert à l'aide d'un fichier json ?

# upsert -> cette commande permet de faire un update si la recherche retourne une valeur, sinon elle insert les données en paramètre.

# exemple d'une synthaxe upsert via bulk sous mongo : Bulk.find(<query>).upsert().update(<update>);
#exempled'une synthaxe avec mongoimport : mongoimport -d nom_db -c nom_collection --mode upsert --upsertFields Id --file "C:/.../movies_upsert.json" --jsonArray

The database exists.
Datas from json document well injected...


In [17]:
################################ Pagination des résultats ################################

# Q : Afficher les 12 documents (films) à partir du dixième inclus en se limitant aux title et _id.
# Indice : Les options skip et limit permettent de « paginer » le résultat (comme en SQL)

def Q6():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_Q6 = [ movie for movie in my_db["movies"].find({},{"_id": 1, "title" : 1}).limit(12).skip(10)]
        print("Movies:\n{}".format(movies_Q6))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q6()

The database exists.
Movies:[{'_id': 'movie:11', 'title': 'PiÃ¨ge de cristal'}, {'_id': 'movie:12', 'title': '58 minutes pour vivre'}, {'_id': 'movie:13', 'title': 'Van Gogh'}, {'_id': 'movie:14', 'title': 'Seven'}, {'_id': 'movie:15', 'title': 'Twelve Monkeys'}, {'_id': 'movie:16', 'title': 'Le last_name de la rose'}, {'_id': 'movie:17', 'title': 'Pulp fiction'}, {'_id': 'movie:18', 'title': 'Mary Ã\xa0 tout prix'}, {'_id': 'movie:19', 'title': 'Terminator'}, {'_id': 'movie:20', 'title': 'Les dents de la mer'}, {'_id': 'movie:21', 'title': 'Le silence des agneaux'}, {'_id': 'movie:22', 'title': 'Godzilla'}]


In [19]:
################################ Trier les resultats ################################
# Q : Trier les documents (de la dernière qry) sur le titre du film ds un ordre croissant (ensuite décroissant).
# Indice : utiliser sort()

def Q7():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_filtered_asc_Q7 = [ movie for movie in my_db["movies"].find({},{"_id": 1, "title" : 1}).limit(12).skip(10).sort("title", 1) ]
        print("Movies asc:\n{}".format(movies_filtered_asc_Q7))
        movies_filtered_desc_Q7 = [ movie for movie in my_db["movies"].find({},{"_id": 1, "title" : 1}).limit(12).skip(10).sort("title", -1) ]
        print("Movies desc:\n{}".format(movies_filtered_desc_Q7))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q7()

The database exists.
Movies asc:[{'_id': 'movie:50', 'title': 'The Matrix Revolutions'}, {'_id': 'movie:90', 'title': 'The Dark Knight Rises'}, {'_id': 'movie:89', 'title': 'The Dark Knight'}, {'_id': 'movie:19', 'title': 'Terminator'}, {'_id': 'movie:70', 'title': 'Taxi driver'}, {'_id': 'movie:62', 'title': 'Stalingrad'}, {'_id': 'movie:47', 'title': 'Spider-Man'}, {'_id': 'movie:68', 'title': 'Soleil vert'}, {'_id': 'movie:6', 'title': 'Sleepy Hollow'}, {'_id': 'movie:74', 'title': 'Skyfall'}, {'_id': 'movie:59', 'title': 'SixiÃ¨me sens'}, {'_id': 'movie:38', 'title': 'Shining'}]
Movies desc:[{'_id': 'movie:37', 'title': 'Eyes Wide Shut'}, {'_id': 'movie:77', 'title': 'Fargo'}, {'_id': 'movie:40', 'title': 'FenÃªtre sur cour'}, {'_id': 'movie:9', 'title': 'Gladiator'}, {'_id': 'movie:22', 'title': 'Godzilla'}, {'_id': 'movie:69', 'title': 'Heat'}, {'_id': 'movie:8', 'title': 'Impitoyable'}, {'_id': 'movie:86', 'title': 'Inception'}, {'_id': 'movie:79', 'title': 'Inglourious Basterds

In [20]:
################################ Critères de recherche simples ################################
# Q : Afficher les infos du film dont l'id est 2, se limiter aux infos relatives au titre, au genre et au summary.
# Indice : utiliser findOne({"_id": "movie:2"}, {"title" : 1,...})
# Projection (ramener des sous parties des documents)

def Q8():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation du film.
        movie_query_Q8 = my_db["movies"].find_one({"_id": "movie:2"},{"title" : 1, "genre": 1, "summary": 1 })
        print("Movie with where id is 2 :\n{}".format(movie_query_Q8))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q8()

The database exists.
Movie with where id is 2 :{'_id': 'movie:2', 'title': 'Alien', 'genre': 'Science-fiction', 'summary': 'PrÃ¨s d\'un vaisseau spatial Ã©chouÃ© sur une lointaine planÃ¨te, des Terriens en mission dÃ©couvrent de bien Ã©tranges "oeufs". Ils en ramÃ¨nent un Ã\xa0 bord, ignorant qu\'ils viennent d\'introduire parmi eux un huitiÃ¨me passager particuliÃ¨rement fÃ©roce et meurtrier. '}


In [15]:
# Q : Combien de films ont été produit en en 1979 et les afficher en se limitant aux infos relatives au titre, 
# à l'année et à la liste des acteurs suivant un ordre alphabétique décroissant sur le titre.
# Indice : 2films : Alien et les bronzés font du ski.

def Q9():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation du film.
        movies_query_Q9 = [ movie for movie in my_db["movies"].find({"year": 1979},{"title" : 1, "year": 1, "actors": 1 }).sort([("title", -1), ("year", 1)]) ]
        print("Movies produced in 1979 :\n{}".format(movies_query_Q9))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
      
Q9()

The database exists.
Movies produced in 1979 :
[{'_id': 'movie:31', 'title': 'Les bronzÃ©s font du ski', 'year': 1979, 'actors': [{'_id': 'artist:75', 'role': 'Nathalie Morin'}, {'_id': 'artist:76', 'role': 'Christiane'}, {'_id': 'artist:84', 'role': 'Jean-Claude Dus'}, {'_id': 'artist:85', 'role': 'JÃ©rÃ´me'}, {'_id': 'artist:86', 'role': 'Popeye'}]}, {'_id': 'movie:2', 'title': 'Alien', 'year': 1979, 'actors': [{'_id': 'artist:5', 'role': 'Ripley'}]}]


In [33]:
# Q : Afficher les infos sur le film dont le genre est "Science-fiction" et produit en 1979 sans préciser l'_id et le summary.
# Indice : Alien ! Si on choisit d’indiquer des attributs à exclure on leur donne la valeur 0 ou null !

def Q10():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation du film.
        movie_query_Q10 = my_db["movies"].find_one({"year": 1979, "genre" : "Science-fiction"},{"_id" : 0, "summary": 0})
        print("SF movies produced in 1979 :\n{}".format(movie_query_Q10))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q10()

The database exists.
SF movies produced in 1979 :
{'title': 'Alien', 'year': 1979, 'genre': 'Science-fiction', 'country': 'USA', 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:5', 'role': 'Ripley'}]}


In [35]:
################################ Critères de recherche complexes ################################

# Recherche dans un objet ou un tableau imbriqué.
# Q : Combien de films produits par le director dont l'id est 4 et les afficher de plus récent au moins récent 
# (sans le résumé ni le country ni le genre).
# Indice : Gladiator, Blade Runer et Alien

def Q11():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q11 = [ movie for movie in my_db["movies"].find({"director":{"_id" : "artist:4"}},{"genre" : 0, "summary": 0, "country": 0}).sort("year", 1) ]
        print("Movies produced by director 4 :\n{}".format(movies_query_Q11))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q11()

The database exists.
Movies produced by director 4 :
[{'_id': 'movie:2', 'title': 'Alien', 'year': 1979, 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:5', 'role': 'Ripley'}]}, {'_id': 'movie:10', 'title': 'Blade Runner', 'year': 1982, 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:24', 'role': 'Deckard'}, {'_id': 'artist:25', 'role': 'Batty'}]}, {'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}]


In [36]:
# Q : Combien de films produits par le director qui porte l'id 4 et dont l'un des acteurs a joué le rôle de Maximus 
# et les afficher sans le résumé ni le country ni le genre.

def Q12():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q12 = [ movie for movie in my_db["movies"].find({"director":{"_id" : "artist:4"}, "actors.role": "Maximus"},{"genre" : 0, "summary": 0, "country": 0}).sort("year", 1) ]
        print("Movies produced by director 4 with an actor who has played in Maximus:\n{}".format(movies_query_Q12))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q12()

The database exists.
Movies produced by director 4 with an actor who has pl:
[{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}]


In [38]:
################################ Critère de recherche par intervalles ################################
# Q : Ts les films produits après 2000 compris et avant 2005 exlus (dans un ordre chronologique) en se limitant au titre 
# du film et son année de production.

def Q13():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q13 = [ movie for movie in my_db["movies"].find({"year":{"$gte":2000,"$lt":2005}},{"title" : 1, "year": 1}).sort("year", 1) ]
        print("Movies produced :\n{}".format(movies_query_Q13))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q13()

The database exists.
Movies produced :
[{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000}, {'_id': 'movie:87', 'title': 'Memento', 'year': 2000}, {'_id': 'movie:62', 'title': 'Stalingrad', 'year': 2001}, {'_id': 'movie:47', 'title': 'Spider-Man', 'year': 2002}, {'_id': 'movie:49', 'title': 'The Matrix reloaded', 'year': 2003}, {'_id': 'movie:50', 'title': 'The Matrix Revolutions', 'year': 2003}, {'_id': 'movie:60', 'title': 'Lost in Translation', 'year': 2003}, {'_id': 'movie:61', 'title': 'Kill Bill', 'year': 2003}]


In [39]:
################################ Opérateurs booléens ################################
# Q : Lister les films parus en 1997 ou avec l'acteur id = 147.
# db.movies.find({$or : [{"year": 1997}, {"actors._id": "artist:147"}] })

def Q14():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q14 = [ movie for movie in my_db["movies"].find({"$or" : [{"year": 1997}, {"actors._id": "artist:147"}] },{"title" : 1, "year": 1, "actors":1}) ]
        print("Movies produced in 1997 or with an actor who has an id = 147 :\n{}".format(movies_query_Q14))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q14()

The database exists.
Movies produced in 1997 or with an actor who has an id = 147 :
[{'_id': 'movie:3', 'title': 'Titanic', 'year': 1997, 'actors': [{'_id': 'artist:109', 'role': 'Rose DeWitt Bukater'}, {'_id': 'artist:110', 'role': 'Jack Dawson'}]}, {'_id': 'movie:5', 'title': 'Volte/Face', 'year': 1997, 'actors': [{'_id': 'artist:11', 'role': 'Sean Archer/Castor Troy'}, {'_id': 'artist:12', 'role': 'Castor Troy/Sean Archer'}]}, {'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}, {'_id': 'movie:26', 'title': 'Les pleins pouvoirs', 'year': 1997, 'actors': [{'_id': 'artist:20', 'role': 'Luther Whitney'}, {'_id': 'artist:21', 'role': 'Le prÃ©sident Richmond'}, {'_id': 'artist:69', 'role': 'Seth Frank'}]}, {'_id': 'movie:28', 'title': 'Le monde perdu', 'year': 1997, 'actors': [{'_id': 'artist:57', 'role':

In [16]:
################################ Opérateurs ensemblistes : in, not in, any, all, exists ################################
# Q : Combien de films dans lesquels ont joué au moins un des artistes suivants : 34, 98 et 1 et afficher leur titre et 
# la liste des actors ; sens du tri : chronologique et alphabétique croissant sur les noms des artistes.
# Indice : Utiliser $in ; 3 films : Le parrain III, Bad Lieutenant, Reservoir dogs, Pulp fiction.

def Q15():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q15 = [ movie for movie in my_db["movies"].find({"actors._id": {"$in": ["artist:34", "artist:98", "artist:1"]}}, {"title": 1, "actors": 1, "date": 1}).sort([("year", 1), ("actors", 1)]) ]
        print("Movies with artist 24, 98 or 1 :\n{}".format(movies_query_Q15))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q15()

The database exists.
Movies with artist 24, 98 or 1 :
[{'_id': 'movie:56', 'title': 'Le parrain III', 'actors': [{'_id': 'artist:1', 'role': 'Mary Corleone'}, {'_id': 'artist:176', 'role': 'Michael Corleone'}, {'_id': 'artist:183', 'role': 'Kay Adams'}, {'_id': 'artist:186', 'role': 'Vincent Mancini'}]}, {'_id': 'movie:36', 'title': 'Reservoir dogs', 'actors': [{'_id': 'artist:37', 'role': 'Mr. Brown'}, {'_id': 'artist:98', 'role': 'Mr. White/Larry'}, {'_id': 'artist:99', 'role': 'Freddy Newendyke/Mr. Orange'}, {'_id': 'artist:100', 'role': 'Nice Guy Eddie'}, {'_id': 'artist:215', 'role': 'Mr Blonde'}, {'_id': 'artist:217', 'role': 'Mr Pink'}, {'_id': 'artist:218', 'role': 'Mr Blue'}]}, {'_id': 'movie:53', 'title': 'Bad Lieutenant', 'actors': [{'_id': 'artist:98', 'role': 'Le lieutenant'}]}, {'_id': 'movie:17', 'title': 'Pulp fiction', 'actors': [{'_id': 'artist:11', 'role': 'Vincent Vega'}, {'_id': 'artist:27', 'role': 'Butch Coolidge'}, {'_id': 'artist:37', 'role': 'Jimmy Dimmick'}, 

In [15]:
# Q : Chercher les films dans lesquels ont joué ensemble les actors 23 et 147 ?
# Indice : Utiliser $all ; Gladiator

def Q16():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q16 = [ movie for movie in my_db["movies"].find({"actors._id": {"$all": ["artist:23", "artist:147"]}}) ]
        print("Movies with actors 23 and 147 :\n{}".format(movies_query_Q16))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q16()

The database exists.
Movies with actors 23 and 147 :
[{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'genre': 'drama', 'summary': "Le gÃ©nÃ©ral romain Maximus est le plus fidÃ¨le                    soutien de l'empereur Marc AurÃ¨le, qu'il a                    conduit de victoire en victoire avec une                    bravoure et un dÃ©vouement exemplaires.                    Jaloux du prestige de Maximus, et plus                    encore de l'amour que lui voue l'empereur,                    le fils de Marc-AurÃ¨le, Commode, s'arroge                    brutalement le pouvoir, puis ordonne                    l'arrestation du gÃ©nÃ©ral et son exÃ©cution.                    Maximus Ã©chappe Ã\xa0 ses assassins mais ne peut                    empÃªcher le massacre de sa famille. CapturÃ©                    par un marchand d'esclaves, il devient                    gladiateur et prÃ©pare sa vengeance.", 'country': 'USA', 'director': {'_id': 'artist:4'}, 'actors': [{'_id': 'artist:

In [17]:
# Q : Combien de films dans lesquels ont joué ni l'acteur 34, ni 98, ni 1 et afficher leur titre et la liste des actors ; 
# sens du tri : chronologique décroissant et alphabétique croissant sur les titres des films.
# Indice : 84 films ; Utiliser $in

def Q17():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q17 = [ movie for movie in my_db["movies"].find({"actors._id": {"$not": {"$in": ["artist:34", "artist:98", "artist:1"]}}}, {"title": 1, "actors": 1}).sort([("year", -1), ("title", 1)]) ]
        print("Movies without actors 34, 98 or 1 :\n{}".format(movies_query_Q17))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q17()

The database exists.
Movies without actors 34, 98 or 1 :
[{'_id': 'movie:92', 'title': 'Nous trois ou rien', 'actors': []}, {'_id': 'movie:85', 'title': 'Interstellar', 'actors': [{'_id': 'artist:267', 'role': 'Cooper'}, {'_id': 'artist:268', 'role': '   Brand '}, {'_id': 'artist:269', 'role': 'Professor Brand'}]}, {'_id': 'movie:78', 'title': 'Django unchained', 'actors': [{'_id': 'artist:110', 'role': 'Calvin Candie'}, {'_id': 'artist:212', 'role': 'Stephen'}, {'_id': 'artist:255', 'role': 'Django'}, {'_id': 'artist:256', 'role': 'Dr. King Schultz'}]}, {'_id': 'movie:90', 'title': 'The Dark Knight Rises', 'actors': [{'_id': 'artist:22', 'role': 'Lucius Fox'}, {'_id': 'artist:116', 'role': 'Commissioner Gordon'}, {'_id': 'artist:268', 'role': 'Selina'}, {'_id': 'artist:269', 'role': 'Alfred'}, {'_id': 'artist:271', 'role': 'Miranda'}, {'_id': 'artist:274', 'role': 'Bruce Wayne / Batman'}]}, {'_id': 'movie:86', 'title': 'Inception', 'actors': [{'_id': 'artist:110', 'role': 'Cobb'}, {'_

In [22]:
# Q : Combien de films qui n’ont pas d’attribut actors ? Combien de films dont l'attribut actors n'est pas renseigné et les afficher ?
# Indice : 0 films ; 6 films

def Q18():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q18 = [ movie for movie in my_db["movies"].find( { "actors._id": { "$exists": 0 } } ) ]
        print("Movies without actors :\n{}".format(movies_query_Q18))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q18()

The database exists.
Movies without actors :
[{'_id': 'movie:4', 'title': 'Sacrifice', 'year': 1986, 'genre': 'drama', 'summary': None, 'country': 'FR', 'director': {'_id': 'artist:9'}, 'actors': []}, {'_id': 'movie:25', 'title': 'Kagemusha', 'year': 1980, 'genre': 'Guerre', 'summary': "Au XVIe siecle, Takeda, grand seigneur trouble par les guerres civiles de son pays, fait appel a un Kagemusha pour l'aider dans ses batailles. Takeda est blesse et avant de mourir, il exige que sa mort soit tenue secrete pendant trois ans afin d'eviter un eclatement du clan. Le Kagemusha devra le remplacer... ", 'country': 'JP', 'director': {'_id': 'artist:68'}, 'actors': []}, {'_id': 'movie:40', 'title': 'FenÃªtre sur cour', 'year': 1954, 'genre': 'Suspense', 'summary': "En repos forcÃ© Ã\xa0 cause d'une jambe plÃ¢trÃ©e, le reporter L.B. Jefferies observe au tÃ©lÃ©objectif les voisins de l'immeuble d'en face. C'est ainsi qu'il remarque le curieux manÃ¨ge d'un reprÃ©sentant de commerce, M. Thorwald, qu'

In [25]:
################################ Jointure ################################

def Q19():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        eastwood = my_db["artists"].find_one({"first_name": "Clint", "last_name": "Eastwood"})
        movies_query_Q19 = [ movie for movie in my_db["movies"].find({"director._id": eastwood['_id']}, {"title": 1, "director._id" : 1}) ]
        print("Movies created by Clint Eastwood :\n{}".format(movies_query_Q19))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q19()

The database exists.
{'_id': 'artist:20', 'last_name': 'Eastwood', 'first_name': 'Clint', 'birth_date': 1930}
Movies created by Clint Eastwood :
[{'_id': 'movie:8', 'title': 'Impitoyable', 'director': {'_id': 'artist:20'}}, {'_id': 'movie:26', 'title': 'Les pleins pouvoirs', 'director': {'_id': 'artist:20'}}, {'_id': 'movie:63', 'title': 'Million Dollar Baby', 'director': {'_id': 'artist:20'}}]


In [31]:
################################ Expressions régulières ################################

# Q : Tous les films dont le titre commence par 'Re'.
# db.movies.find ({"title": /^Re/}, {"actors": null, "summary": 0}).pretty()
def Q20():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q20 = [ movie for movie in my_db["movies"].find({"title": { "$regex": '^Re' }}, {"actors": 0, "summary": 0}) ]
        print("Movies with title begining by re:\n{}".format(movies_query_Q20))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q20()

The database exists.
Movies with title begining :
[{'_id': 'movie:36', 'title': 'Reservoir dogs', 'year': 1992, 'genre': 'crime', 'country': 'USA', 'director': {'_id': 'artist:37'}}]


In [32]:
# Q : Tous les films dont le titre finit par 'la'.
# db.movies.find ({"title": /la$/}, {"actors": null, "summary": 0}).pretty()
def Q21():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q21 = [ movie for movie in my_db["movies"].find({"title": { "$regex": 'la$' }}, {"actors": 0, "summary": 0}) ]
        print("Movies with title ending by la :\n{}".format(movies_query_Q21))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q21()

The database exists.
Movies with title ending by la :
[{'_id': 'movie:22', 'title': 'Godzilla', 'year': 1998, 'genre': 'Action', 'country': 'USA', 'director': {'_id': 'artist:58'}}]


In [3]:
# Q : Combien de films ont été joués par l'acteur dont l'id est 147 et les afficher en se limitant au titre du film et 
# la liste des acteurs.

def Q22():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_count_Q22 = my_db["movies"].count_documents(({"actors._id": "artist:147"}))
        print("Actor 147 has played in {} movies.".format(movies_count_Q22))
        movies_query_Q22 = [ movie for movie in my_db["movies"].find({"actors._id": "artist:147"}, {"title": 1, "actors": 1}) ]
        
        print("Movie with actor 147 :\n{}".format(movies_query_Q22))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q22()

The database exists.
Actor 147 has played in 1 movies.
Movie with actor 147 :
[{'_id': 'movie:9', 'title': 'Gladiator', 'actors': [{'_id': 'artist:23', 'role': 'Maximus'}, {'_id': 'artist:147', 'role': 'Commode'}, {'_id': 'artist:148', 'role': 'Lucilla'}, {'_id': 'artist:149', 'role': 'Marc Aurele'}]}]


In [5]:
# Q : Retourner les films joués par l'acteur 147 sans les acteurs et sans le résumé.

def Q23():
    #Connexion à mongo...
    my_client = get_client()
    
    #Connexion à la base de données si elle existe.
    if "movies_artists" in my_client.list_database_names():
        print("The database exists.")
        my_db = my_client["movies_artists"]
        #Récuperation des films.
        movies_query_Q23 = [ movie for movie in my_db["movies"].find({"actors._id": "artist:147"}, {"summary": 0, "actors": 0}) ]
        
        print("Movie with actor 147 :\n{}".format(movies_query_Q23))
        
    else:
        print("The database doesn't exist...You should create it by calling previous methods")
        
Q23()

The database exists.
Movie with actor 147 :
[{'_id': 'movie:9', 'title': 'Gladiator', 'year': 2000, 'genre': 'drama', 'country': 'USA', 'director': {'_id': 'artist:4'}}]
